# 🗺️ Bangkok Taxi Analysis - Interactive Maps

This notebook creates interactive maps for your taxi analysis results.

**No complex dependencies needed!** Just install Folium:
```bash
pip3 install folium pandas
```

Then run all cells below to generate interactive HTML maps.

## Step 1: Import Libraries

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
import glob

print("✅ Libraries imported successfully!")

## Step 2: Load Data

In [ ]:
# Load night demand zones
demand_files = glob.glob("output_night_demand_zones/part-*.csv")
demand = pd.concat([pd.read_csv(f) for f in demand_files])
demand_bangkok = demand[(demand['lat_bin'] > 13.5) & (demand['lat_bin'] < 14.0) &
                        (demand['lon_bin'] > 100.3) & (demand['lon_bin'] < 100.8)]

# Load taxi deserts
desert_files = glob.glob("output_taxi_deserts/part-*.csv")
deserts = pd.concat([pd.read_csv(f) for f in desert_files])

print(f"✅ Loaded {len(demand_bangkok):,} night demand zones (Bangkok)")
print(f"✅ Loaded {len(deserts):,} taxi desert zones")
print(f"\nTop 5 demand zones:")
print(demand_bangkok.nlargest(5, 'demand_supply_ratio')[['lat_bin', 'lon_bin', 'occupied_count', 'demand_supply_ratio']])

## Map 1: Night Demand Zones 🌃

Shows areas with high taxi demand at night (20:00-02:00).
- **Red circles** = Extreme demand (ratio > 100)
- **Orange circles** = High demand
- **Circle size** = Number of occupied taxis

In [ ]:
# Create map centered on Bangkok
m1 = folium.Map(location=[13.75, 100.52], zoom_start=12, tiles='OpenStreetMap')

# Add demand zones as circles
for _, row in demand_bangkok.nlargest(100, 'demand_supply_ratio').iterrows():
    color = 'red' if row['demand_supply_ratio'] > 100 else 'orange'
    folium.CircleMarker(
        location=[row['lat_bin'], row['lon_bin']],
        radius=min(row['occupied_count']/30, 15),
        color=color,
        fill=True,
        fillOpacity=0.6,
        popup=f"""
            <b>Night Demand Zone</b><br>
            📍 Location: {row['lat_bin']:.3f}, {row['lon_bin']:.3f}<br>
            🚖 Occupied: {row['occupied_count']}<br>
            🚕 Available: {row['available_count']}<br>
            📊 Demand Ratio: {row['demand_supply_ratio']:.0f}
        """,
        tooltip=f"Demand: {row['demand_supply_ratio']:.0f}"
    ).add_to(m1)

# Save and display
m1.save('map1_night_demand.html')
print("✅ Saved: map1_night_demand.html")
m1

## Map 2: Taxi Deserts 🚕

Shows areas with taxis waiting but few passengers (22:00-04:00).
- **Purple circles** = Desert zones (high % of unhired taxis)
- **Circle size** = Number of unhired taxis

In [ ]:
m2 = folium.Map(location=[13.75, 100.52], zoom_start=11, tiles='CartoDB positron')

# Add desert zones
for _, row in deserts.iterrows():
    folium.CircleMarker(
        location=[row['lat_bin'], row['lon_bin']],
        radius=min(row['unhired_count']/5, 12),
        color='purple',
        fill=True,
        fillOpacity=0.7,
        popup=f"""
            <b>Taxi Desert Zone</b><br>
            📍 Location: {row['lat_bin']:.3f}, {row['lon_bin']:.3f}<br>
            🚕 Unhired Taxis: {row['unhired_count']}<br>
            📊 Total Activity: {row['total_count']}<br>
            ⚠️ Desert Score: {row['desert_score']*100:.1f}%
        """,
        tooltip=f"Desert: {row['desert_score']*100:.0f}%"
    ).add_to(m2)

m2.save('map2_taxi_deserts.html')
print("✅ Saved: map2_taxi_deserts.html")
m2

## Map 3: Combined View with Layer Control 🗺️

Toggle between demand zones and taxi deserts.

In [ ]:
m3 = folium.Map(location=[13.75, 100.52], zoom_start=12)

# Add demand zones layer
demand_layer = folium.FeatureGroup(name='🌃 Night Demand Zones', show=True)
for _, row in demand_bangkok.nlargest(50, 'demand_supply_ratio').iterrows():
    folium.CircleMarker(
        location=[row['lat_bin'], row['lon_bin']],
        radius=8,
        color='red',
        fill=True,
        fillOpacity=0.5,
        popup=f"Demand: {row['demand_supply_ratio']:.0f}"
    ).add_to(demand_layer)
demand_layer.add_to(m3)

# Add desert zones layer
desert_layer = folium.FeatureGroup(name='🚕 Taxi Deserts', show=True)
for _, row in deserts.nlargest(30, 'desert_score').iterrows():
    folium.CircleMarker(
        location=[row['lat_bin'], row['lon_bin']],
        radius=6,
        color='blue',
        fill=True,
        fillOpacity=0.5,
        popup=f"Desert: {row['desert_score']*100:.0f}%"
    ).add_to(desert_layer)
desert_layer.add_to(m3)

# Add layer control
folium.LayerControl(collapsed=False).add_to(m3)

m3.save('map3_combined.html')
print("✅ Saved: map3_combined.html")
m3

## 📁 Output Files

The maps are saved as HTML files in your current directory:
- `map1_night_demand.html` - Night demand zones
- `map2_taxi_deserts.html` - Taxi desert zones  
- `map3_combined.html` - Combined view with layer control

**To view:** Double-click any HTML file to open in your browser!